# LSML1 SGA

> Large Scale Machine Learning 1 (Spring23)

> Sergey Terskov

# Solution 1. Spark SQL

In [5]:
%%time

#Initialize Spark
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StringType

sc = SparkContext(appName="LSML1_SGA")
se = SparkSession(sc)

# Read data from HDFS
events_df = se.read.format("csv") \
      .options(delimiter="\t", header=True) \
      .schema("user_id bigint, session_id bigint, event_type string, event_page string, timestamp bigint") \
      .load("hdfs:///data//clickstream.csv")

print("Initial dataset example:")
events_df.show(5)

events_df.createOrReplaceTempView("events")

# SQL-query for routes
query = """
    WITH filtered_events AS (
        SELECT user_id, session_id, event_page, 
               ROW_NUMBER() OVER (PARTITION BY user_id, session_id ORDER BY timestamp) as row_num,
               SUM(CASE WHEN event_type LIKE '%error%' THEN 1 ELSE 0 END) OVER (PARTITION BY user_id, session_id) as has_error
        FROM events
        WHERE event_type = 'page'
    ),
    final_routes AS (
        SELECT user_id, session_id,
               COLLECT_LIST(event_page) AS route
        FROM filtered_events
        WHERE has_error = 0 OR row_num <= (SELECT MAX(row_num) FROM filtered_events fe WHERE fe.user_id = filtered_events.user_id AND fe.session_id = filtered_events.session_id AND fe.has_error > 0)
        GROUP BY user_id, session_id
    ),
    route_counts AS (
        SELECT CONCAT_WS('-', route) AS route, COUNT(*) AS count
        FROM final_routes
        GROUP BY route
    )
    SELECT route, count
    FROM route_counts
    ORDER BY count DESC
    LIMIT 30
"""

# Execute SQL
top_routes_df = se.sql(query)

# Show result
print("Top 30 frequent routes:")
top_routes_df.show(30)

# Save to csv
top_routes_df.toPandas().to_csv("solution_1_SQL.csv", sep="\t")

2024-10-30 20:07:18,501 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Initial dataset example:


+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
+-------+----------+------------+----------+----------+
only showing top 5 rows

Top 30 frequent routes:


+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 5984|
|        main-archive|  839|
|         main-rabota|  804|
|       main-internet|  687|
|          main-bonus|  657|
|           main-news|  594|
|        main-tariffs|  511|
|         main-online|  445|
|          main-vklad|  408|
| main-archive-rabota|  136|
| main-rabota-archive|  134|
|   main-rabota-bonus|  111|
|  main-bonus-archive|  109|
|    main-news-rabota|  108|
|   main-bonus-rabota|  105|
|main-internet-arc...|  104|
|main-internet-rabota|  104|
|   main-archive-news|  103|
|    main-rabota-news|  103|
|  main-archive-bonus|  100|
|main-archive-inte...|  100|
|   main-news-archive|   94|
|main-rabota-internet|   92|
|main-tariffs-inte...|   89|
| main-internet-bonus|   85|
|  main-internet-news|   81|
|     main-news-bonus|   80|
|  main-news-internet|   78|
|main-archive-tariffs|   76|
| main-archive-online|   76|
+--------------------+-----+



CPU times: user 225 ms, sys: 121 ms, total: 346 ms
Wall time: 1min 4s
